In [64]:
from joblib import Parallel, delayed
from itertools import combinations
from myf import calculate_g_xt, calculate_f_jk

def calculate_single_motif_parallel(profiles, gamma):
    """前面所有dtw可以一次算完之後存起來
    存法可以先考慮用numpy array, like
    dtw_list = [
    [infinite, dtw(0,1), dtw(0,2)...]
    [infinite, infinite, dtw(1,2), dtw(1,3)...]
    ...
    ]
    以matrix表示，應該會是一個diagnal 和下半部 all infinite的上三角矩陣
    這邊可以整個獨立出來在一開始就算好
    """
    # 生成所有轮廓对组合
    pairs = list(combinations(enumerate(profiles), 2))

    # 并行计算所有对的DTW距离
    distances = Parallel(n_jobs=-1)(
        delayed(calculate_f_jk)(dp1, dp2) for (i, dp1), (j, dp2) in pairs
    )
    """上面先存起來，之後就不用一直重算"""

    """if above is numpy array with lower triangle and diagnol all infinite, can use built in numpy min to find the minumum"""
    """for filter out to_remove profiles, you can use a bool array to mark the profiles to be removed, 
    and then use it as numpy mask index to filter them out in one go"""
    # 找到最小距离对
    min_distance = float("inf")
    best_pair = None
    for idx, ((i, dp1), (j, dp2)) in enumerate(pairs):
        distance = distances[idx]
        if distance < min_distance and distance < gamma:
            min_distance = distance
            best_pair = (i, j)

    if best_pair is None:
        return None

    i, j = best_pair
    dpx, dpy = profiles[i], profiles[j]
    motif = [dpx, dpy]
    return {"motif": motif, "pair_idx": (i, j), "score": round(min_distance, 2)}


def calculate_all_motifs_parallel(profiles, gamma, tau):
    motifs = []
    profiles = [tuple(dp) for dp in profiles]
    remaining_profiles = profiles.copy()

    while True:
        result = calculate_single_motif_parallel(remaining_profiles, gamma)
        if result is None:
            break

        motif_info = {
            "idx": len(motifs) + 1,
            "motif": result["motif"],
            "pair_idx": result["pair_idx"],
            "score": result["score"],
        }
        motifs.append(motif_info)
        i, j = result["pair_idx"]
        dpx, dpy = remaining_profiles[i], remaining_profiles[j]
        min_distance = result["score"]

        # 并行检查需要移除的轮廓
        def should_remove(idx, dp):
            if idx == i or idx == j:
                return True
            return (
                calculate_f_jk(dpx, dp) <= min_distance + tau
                or calculate_f_jk(dpy, dp) <= min_distance + tau
            )

        to_remove = Parallel(n_jobs=-1)(
            delayed(should_remove)(idx, dp) for idx, dp in enumerate(remaining_profiles)
        )
        """可以改用bool array紀錄是否已被移除，array的index會等於前面matrix的row index
        這樣如果有一個dp被移除，找下一個motif時就可以直接用numpy array的mask index來過濾掉"""
        remaining_profiles = [
            dp for idx, dp in enumerate(remaining_profiles) if not to_remove[idx]
        ]

    return motifs

In [65]:
if __name__ == "__main__":
    profiles = [
        (1.2, 3.4, 2.1),
        (1.1, 3.5, 2.0),
        (5.6, 4.2, 6.7),
        (5.5, 4.3, 6.6),
        (10.1, 12.3, 11.2),
    ]
    profiles = [
        (100, 120, 115),  # dp0
        (100, 120, 110),  # dp1
        (110, 110, 110),  # dp2
        (140, 60, 194),  # dp3
        (150, 50, 200),  # dp4
    ]
    gamma = 20.0  # 最大允许距离阈值
    tau = 1.0  # 容忍值

    # 并行计算
    motifs = calculate_all_motifs_parallel(profiles, gamma, tau)

    # 打印结果
    for m in motifs:
        print(f"Motif {m['idx']}: Score={m['score']}, Pair={m['pair_idx']}")
        print(f"  dp1: {m['motif'][0]}")
        print(f"  dp2: {m['motif'][1]}\n")

/opt/miniconda3/envs/nthu_ml/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
/opt/miniconda3/envs/nthu_ml/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
/opt/miniconda3/envs/nthu_ml/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
/opt/miniconda3/envs/nthu_ml/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
/opt/miniconda3/envs/nthu_ml/lib/python3.10/site-packages/tslearn/bases/bases.py:15:

Motif 1: Score=0.05, Pair=(0, 1)
  dp1: (100, 120, 115)
  dp2: (100, 120, 110)

Motif 2: Score=0.24, Pair=(0, 1)
  dp1: (140, 60, 194)
  dp2: (150, 50, 200)



In [66]:
all_dpw = []

In [100]:
profiles = [
        (100, 120, 115),  # dp0
        (100, 120, 110),  # dp1
        (110, 110, 110),  # dp2
        (140, 60, 194),  # dp3
        (150, 50, 200),  # dp4
    ]
pairs = list(combinations(enumerate(profiles), 2))
distances = Parallel(n_jobs=-1)(
        delayed(calculate_f_jk)(dp1, dp2) for (i, dp1), (j, dp2) in pairs
    )

In [68]:
print(list(enumerate(profiles)))
profiles2 = list(enumerate(profiles))

[(0, (100, 120, 115)), (1, (100, 120, 110)), (2, (110, 110, 110)), (3, (140, 60, 194)), (4, (150, 50, 200))]


In [181]:
import numpy as np

def precompute_distances(profiles, n_jobs=-1,):
    n = len(profiles)
    # initial the matrix
    dtw_matrix = np.full((n, n), np.inf)
    indices = list(combinations(range(n), 2))
    
    # Calculate the score between 2 signals
    distances = Parallel(n_jobs=n_jobs)(
        delayed(calculate_f_jk)(
            profiles[i], profiles[j]
        ) for i,j in indices
    )
    
    # Assign
    for (i, j), dist in zip(indices, distances):
        dtw_matrix[i, j] = dist

    masked_arr = np.ma.masked_where(np.isinf(dtw_matrix), dtw_matrix)
    return masked_arr

def single_motif(masked_arr,id, gamma=2):
    min_value = masked_arr.min()
    if min_value < gamma:
        min_index = np.unravel_index(masked_arr.argmin(), masked_arr.shape)
        row, col = min_index
        masked_arr[row, :] = np.inf 
        masked_arr[:, col] = np.inf  

        motif = {
            "pattern1": profiles[row],
            "pattern2": profiles[col],
            # "average" : avg
        }
        print(motif)
        motif_info = {
            "idx": id,
            "motif": motif,
            "pair_idx": (row,col),
            "score": min_value,
        }

        return motif_info,masked_arr
    else: return None,masked_arr

def all_motifs(profiles,gamma,tau):

    motifs = []
    
    dtw_matrix = precompute_distances(profiles)

    motif, masked_arr = single_motif(dtw_matrix,1,gamma)
    while(True):
        motif, masked_arr = single_motif(masked_arr, len(motifs)+1,gamma)
        if motif is None:
            break

        motifs.append(motif)
        row, col = motif["pair_idx"]
        min_score = motif["score"]

        for i,dpx in enumerate(masked_arr[row]):
            
            if dpx < min_score + tau:
                print(i)
                masked_arr[i, :] = np.inf
                masked_arr[:, i] = np.inf 
        for j,dpy in enumerate(masked_arr[col]):
            if dpy < min_score + tau:
                masked_arr[j, :] = np.inf
                masked_arr[:, j] = np.inf 
    return motifs, masked_arr


In [185]:
motifs,masked_arr = all_motifs(profiles,1,0.5)

{'pattern1': (100, 120, 115), 'pattern2': (100, 120, 110)}
[[inf inf inf inf inf]
 [-- inf 0.15929841724948007 1.121790831644352 1.3459434653714857]
 [-- inf -- 1.0667828620865367 1.2771312107618442]
 [-- inf -- -- 0.24098883878812788]
 [-- inf -- -- --]]
{'pattern1': (100, 120, 110), 'pattern2': (110, 110, 110)}
[[inf inf inf inf inf]
 [inf inf inf inf inf]
 [-- inf inf 1.0667828620865367 1.2771312107618442]
 [-- inf inf -- 0.24098883878812788]
 [-- inf inf -- --]]
{'pattern1': (140, 60, 194), 'pattern2': (150, 50, 200)}
[[inf inf inf inf inf]
 [inf inf inf inf inf]
 [-- inf inf 1.0667828620865367 inf]
 [inf inf inf inf inf]
 [-- inf inf -- inf]]


In [186]:
print(masked_arr)

[[inf inf inf inf inf]
 [inf inf inf inf inf]
 [-- inf inf 1.0667828620865367 inf]
 [inf inf inf inf inf]
 [-- inf inf -- inf]]


In [ ]:
"""
then for any s
if masked_arr[s, i] or arr[i, s] <= min_distance + tau
or masked_arr[s, j] or arr[j, s] <= min_distance + tau
then make s belongs to this motif and update masekd_arr[s] and masekd_arr[[s]] to inf
"""